In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ecommerce_lr").getOrCreate()

data = spark.read.csv("Data\Ecommerce_Customers.csv", inferSchema = True, header = True)

# observe structure and some data
data.printSchema()
data.show(5)

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [11]:
# print first line values
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [9]:
# Data needs to be in the form of ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
# assembling feature part:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

# features column are added to orig data
output = assembler.transform(data)
output.show(5)
output.select("features").show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [15]:
# create final input data to model
final_data = output.select("features",'Yearly Amount Spent')

# split to training and testing
train_data,test_data = final_data.randomSplit([0.7,0.3])

train_data.describe().show(5)
test_data.describe().show(5)

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                351|
|   mean| 493.51483762821044|
| stddev|  77.27367059784397|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                149|
|   mean|  512.9752424281446|
| stddev|  82.59259181523633|
|    min|  314.4385182951061|
|    max|  765.5184619388373|
+-------+-------------------+



In [18]:
from pyspark.ml.regression import LinearRegression
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

test_results = lrModel.evaluate(test_data)

# show results
test_results.residuals.show(5)

Coefficients: [25.36552604934562,38.49366594744958,0.9449831144416184,61.61160616543016] Intercept: -1055.730594773079
+------------------+
|         residuals|
+------------------+
|10.299866237519495|
|-4.341244178505065|
|6.3678987861014775|
|-8.102985967794666|
|10.049005327440454|
+------------------+
only showing top 5 rows



In [19]:
# prediction (using test data for now)
unlabeled_data = test_data.select('features')
predictions = lrModel.transform(unlabeled_data)
predictions.show(5)

print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.7377203726281...| 451.4808759587104|
|[30.8794843441274...|494.54784416335974|
|[30.9716756438877...|488.27071097079124|
|[31.1280900496166...| 565.3556727148493|
|[31.1695067987115...|417.30752547485235|
+--------------------+------------------+
only showing top 5 rows

RMSE: 10.44617321854849
MSE: 109.12253491191971
